# Task 6.5: Profiling Workshop - Find and Fix Bottlenecks

**Module:** 6 - Deep Learning with PyTorch  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate-Advanced)

---

## Learning Objectives

By the end of this notebook, you will:
- [ ] Use PyTorch Profiler to analyze training performance
- [ ] Generate and interpret Chrome trace files
- [ ] Identify CPU, GPU, and data loading bottlenecks
- [ ] Apply optimizations based on profiling results
- [ ] Monitor memory usage during training

---

## Prerequisites

- Completed: Tasks 6.1-6.4
- Knowledge of: Training loops, GPU operations

---

## Real-World Context

Training neural networks is expensive! Profiling helps you:
- **Find bottlenecks**: Is it CPU, GPU, or I/O bound?
- **Optimize resource usage**: Use all your hardware effectively
- **Reduce training cost**: Faster training = less compute cost
- **Debug performance issues**: Why did training suddenly slow down?

At scale, even 10% speedup can save thousands of dollars!

---

## ELI5: What is Profiling?

> **Imagine you're watching a relay race...** 🏃
>
> Each runner (operation) takes time. The race (training) is only as fast as the slowest runner.
>
> **Profiling is like using a stopwatch on each runner:**
> - "The handoff is taking too long!" (data loading)
> - "Runner 3 is really slow!" (GPU bottleneck)
> - "Everyone's waiting for Runner 2!" (CPU-GPU sync)
>
> Once you know who's slow, you can coach them to be faster!
>
> **In AI terms:** Profiling measures exactly how long each operation takes, showing you where to focus optimization efforts.

---

## Part 1: Environment Setup

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torch.profiler import profile, ProfilerActivity, tensorboard_trace_handler, schedule

import time
import os
from pathlib import Path

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Create a model with various layer types for profiling
class ProfilingModel(nn.Module):
    """
    A model designed to demonstrate different operation types in profiling.
    
    Includes convolutions, batch norms, linear layers, and attention-like operations.
    """
    
    def __init__(self, num_classes: int = 10):
        super().__init__()
        
        # Convolutional layers (GPU-heavy)
        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )
        
        # Linear layers
        self.fc_block = nn.Sequential(
            nn.Linear(256 * 8 * 8, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes),
        )
    
    def forward(self, x):
        x = self.conv_block(x)
        x = x.view(x.size(0), -1)
        x = self.fc_block(x)
        return x

model = ProfilingModel(10).to(device)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Load CIFAR-10
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform
)

# Create data loaders with different configurations
BATCH_SIZE = 64

# Suboptimal loader (for demonstration)
slow_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

# Optimized loader
fast_loader = DataLoader(
    trainset, batch_size=BATCH_SIZE, shuffle=True, 
    num_workers=4, pin_memory=True, persistent_workers=True
)

print(f"Training samples: {len(trainset):,}")
print(f"Batch size: {BATCH_SIZE}")

---

## Part 2: Basic Profiling

Let's start with simple profiling to understand the basics.

In [ ]:
# Simple timing profile
def profile_training_loop(model, dataloader, num_batches=50):
    """
    Profile a training loop and return timing breakdown.
    """
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
    timings = {
        'data_loading': [],
        'forward': [],
        'backward': [],
        'optimizer': [],
    }
    
    data_iter = iter(dataloader)
    
    for i in range(num_batches):
        # Data loading
        t0 = time.time()
        try:
            inputs, labels = next(data_iter)
        except StopIteration:
            data_iter = iter(dataloader)
            inputs, labels = next(data_iter)
        inputs, labels = inputs.to(device), labels.to(device)
        if device.type == 'cuda':
            torch.cuda.synchronize()
        timings['data_loading'].append(time.time() - t0)
        
        # Forward
        t0 = time.time()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        if device.type == 'cuda':
            torch.cuda.synchronize()
        timings['forward'].append(time.time() - t0)
        
        # Backward
        t0 = time.time()
        optimizer.zero_grad()
        loss.backward()
        if device.type == 'cuda':
            torch.cuda.synchronize()
        timings['backward'].append(time.time() - t0)
        
        # Optimizer step
        t0 = time.time()
        optimizer.step()
        if device.type == 'cuda':
            torch.cuda.synchronize()
        timings['optimizer'].append(time.time() - t0)
    
    # Calculate averages (skip warmup)
    warmup = 5
    return {k: sum(v[warmup:]) / len(v[warmup:]) * 1000 for k, v in timings.items()}

# Profile with slow loader
print("=== Profiling with num_workers=0 ===")
slow_timings = profile_training_loop(model, slow_loader)
for name, time_ms in slow_timings.items():
    print(f"  {name:15s}: {time_ms:.2f} ms")
print(f"  {'TOTAL':15s}: {sum(slow_timings.values()):.2f} ms")

# Profile with fast loader
print("\n=== Profiling with num_workers=4 ===")
fast_timings = profile_training_loop(model, fast_loader)
for name, time_ms in fast_timings.items():
    print(f"  {name:15s}: {time_ms:.2f} ms")
print(f"  {'TOTAL':15s}: {sum(fast_timings.values()):.2f} ms")

# Speedup
speedup = sum(slow_timings.values()) / sum(fast_timings.values())
print(f"\nSpeedup from optimized data loading: {speedup:.2f}x")

---

## Part 3: PyTorch Profiler Deep Dive

The PyTorch Profiler provides detailed information about:
- CPU and GPU operations
- Memory allocation
- Kernel launches
- Data transfer between CPU and GPU

In [ ]:
def training_step(model, inputs, labels, criterion, optimizer):
    """Single training step for profiling."""
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

# Setup
model = ProfilingModel(10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Profile with PyTorch Profiler
print("Running PyTorch Profiler...")

with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    record_shapes=True,
    profile_memory=True,
    with_stack=True,
) as prof:
    # Run a few training steps
    data_iter = iter(fast_loader)
    for i in range(20):
        inputs, labels = next(data_iter)
        inputs, labels = inputs.to(device), labels.to(device)
        training_step(model, inputs, labels, criterion, optimizer)

# Print summary table
print("\n=== Top Operations by CUDA Time ===")
print(prof.key_averages().table(
    sort_by="cuda_time_total", 
    row_limit=15
))

In [ ]:
# Print summary by operation type
print("\n=== Top Operations by CPU Time ===")
print(prof.key_averages().table(
    sort_by="cpu_time_total", 
    row_limit=15
))

In [ ]:
# Memory profiling
print("\n=== Operations with Memory Allocation ===")
print(prof.key_averages().table(
    sort_by="self_cuda_memory_usage", 
    row_limit=10
))

---

## Part 4: Generating Chrome Trace

Chrome traces provide a visual timeline of operations.

In [ ]:
# Create output directory for traces
trace_dir = Path('./profiler_traces')
trace_dir.mkdir(exist_ok=True)

# Profile with trace export
print("Generating Chrome trace...")

with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    record_shapes=True,
    profile_memory=True,
    with_stack=True,
) as prof:
    data_iter = iter(fast_loader)
    for i in range(20):
        inputs, labels = next(data_iter)
        inputs, labels = inputs.to(device), labels.to(device)
        training_step(model, inputs, labels, criterion, optimizer)

# Export Chrome trace
trace_path = trace_dir / 'training_trace.json'
prof.export_chrome_trace(str(trace_path))
print(f"Chrome trace saved to: {trace_path}")
print("\nTo view the trace:")
print("1. Open Chrome browser")
print("2. Go to chrome://tracing")
print("3. Click 'Load' and select the JSON file")

---

## Part 5: Using Profiler Schedule

For long training runs, use a schedule to profile specific steps.

In [ ]:
# Profiler with schedule
def trace_handler(p):
    """Custom trace handler to save traces."""
    output = p.key_averages().table(sort_by="cuda_time_total", row_limit=5)
    print(f"\n=== Step {p.step_num} ===")
    print(output)

# Use schedule: wait 2 steps, warmup 2 steps, active 3 steps, repeat
prof_schedule = schedule(
    wait=2,      # Don't profile first 2 steps
    warmup=2,    # Profile but don't record next 2 steps
    active=3,    # Actually record next 3 steps
    repeat=2,    # Repeat this cycle 2 times
)

print("Running scheduled profiling...")

with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    schedule=prof_schedule,
    on_trace_ready=trace_handler,
    record_shapes=True,
    with_stack=True,
) as prof:
    data_iter = iter(fast_loader)
    for i in range(20):
        inputs, labels = next(data_iter)
        inputs, labels = inputs.to(device), labels.to(device)
        training_step(model, inputs, labels, criterion, optimizer)
        prof.step()  # Important: signal end of step

---

## Part 6: Memory Profiling

Understanding memory usage is crucial, especially on GPU.

In [ ]:
def print_memory_stats():
    """Print current GPU memory usage."""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1e9
        reserved = torch.cuda.memory_reserved() / 1e9
        max_allocated = torch.cuda.max_memory_allocated() / 1e9
        print(f"Allocated: {allocated:.2f} GB | Reserved: {reserved:.2f} GB | Peak: {max_allocated:.2f} GB")

# Reset memory stats
torch.cuda.reset_peak_memory_stats() if torch.cuda.is_available() else None
torch.cuda.empty_cache() if torch.cuda.is_available() else None

print("=== Memory Usage During Training ===")

# Create fresh model
model = ProfilingModel(10).to(device)
print("After model creation:")
print_memory_stats()

# Forward pass
inputs = torch.randn(BATCH_SIZE, 3, 32, 32, device=device)
labels = torch.randint(0, 10, (BATCH_SIZE,), device=device)
outputs = model(inputs)
print("\nAfter forward pass:")
print_memory_stats()

# Loss computation
criterion = nn.CrossEntropyLoss()
loss = criterion(outputs, labels)
print("\nAfter loss computation:")
print_memory_stats()

# Backward pass
loss.backward()
print("\nAfter backward pass:")
print_memory_stats()

# Cleanup
del outputs, loss
torch.cuda.empty_cache()
print("\nAfter cleanup:")
print_memory_stats()

In [ ]:
# Profile memory allocation per layer
def profile_model_memory(model, input_shape):
    """
    Profile memory usage for each part of the model.
    """
    memory_by_layer = {}
    
    def hook_fn(name):
        def hook(module, input, output):
            if torch.cuda.is_available():
                torch.cuda.synchronize()
                memory_by_layer[name] = {
                    'output_size': output.element_size() * output.nelement() / 1e6,  # MB
                    'output_shape': tuple(output.shape),
                }
        return hook
    
    # Register hooks
    hooks = []
    for name, module in model.named_modules():
        if len(list(module.children())) == 0:  # Leaf modules only
            hooks.append(module.register_forward_hook(hook_fn(name)))
    
    # Run forward pass
    model.eval()
    with torch.no_grad():
        x = torch.randn(*input_shape, device=device)
        _ = model(x)
    
    # Remove hooks
    for h in hooks:
        h.remove()
    
    return memory_by_layer

# Profile our model
memory_profile = profile_model_memory(model, (1, 3, 32, 32))

print("\n=== Memory per Layer (Single Sample) ===")
total_mb = 0
for name, info in sorted(memory_profile.items()):
    print(f"{name:40s} | Shape: {str(info['output_shape']):20s} | Size: {info['output_size']:.2f} MB")
    total_mb += info['output_size']
print(f"{'TOTAL':40s} | {'':20s} | Size: {total_mb:.2f} MB")
print(f"\nFor batch_size={BATCH_SIZE}: ~{total_mb * BATCH_SIZE:.2f} MB")

---

## Part 7: Common Bottlenecks and Solutions

Let's identify and fix common performance issues.

In [ ]:
# Bottleneck 1: Data loading on CPU thread
print("=== Bottleneck 1: Data Loading ===")

def benchmark_data_loading(num_workers_list=[0, 2, 4, 8]):
    results = {}
    
    for num_workers in num_workers_list:
        # Clean up memory between benchmarks
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
        import gc; gc.collect()
        
        loader = DataLoader(
            trainset, batch_size=BATCH_SIZE, shuffle=True,
            num_workers=num_workers, pin_memory=True if num_workers > 0 else False
        )
        
        # Time data loading
        start = time.time()
        for i, (inputs, labels) in enumerate(loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            if i >= 50:
                break
        
        # Sync before measuring time
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        elapsed = time.time() - start
        
        results[num_workers] = elapsed
        print(f"num_workers={num_workers}: {elapsed:.2f}s for 50 batches")
    
    return results

data_loading_results = benchmark_data_loading([0, 2, 4])
best_workers = min(data_loading_results, key=data_loading_results.get)
print(f"\nOptimal num_workers: {best_workers}")

In [ ]:
# Bottleneck 2: Unnecessary CPU-GPU synchronization
print("\n=== Bottleneck 2: CPU-GPU Synchronization ===")

# Bad: Printing loss every iteration (forces sync)
def train_with_frequent_sync(model, loader, num_batches=50):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    
    start = time.time()
    data_iter = iter(loader)
    for i in range(num_batches):
        inputs, labels = next(data_iter)
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # BAD: This forces CPU-GPU sync every iteration!
        _ = loss.item()
    
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    return time.time() - start

# Good: Accumulate loss, sync less often
def train_with_reduced_sync(model, loader, num_batches=50, log_interval=10):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    
    start = time.time()
    running_loss = 0.0
    data_iter = iter(loader)
    
    for i in range(num_batches):
        inputs, labels = next(data_iter)
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss  # Keep as tensor (no sync)
        
        # Only sync occasionally
        if (i + 1) % log_interval == 0:
            avg_loss = running_loss.item() / log_interval
            running_loss = 0.0
    
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    return time.time() - start

# Clean up before benchmarks
import gc
torch.cuda.empty_cache() if torch.cuda.is_available() else None
gc.collect()

# Benchmark
model_copy = ProfilingModel(10).to(device)
time_frequent = train_with_frequent_sync(model_copy, fast_loader)

# Clean up between benchmarks
del model_copy
torch.cuda.empty_cache() if torch.cuda.is_available() else None
gc.collect()

model_copy = ProfilingModel(10).to(device)  # Fresh model
time_reduced = train_with_reduced_sync(model_copy, fast_loader)

# Clean up
del model_copy
torch.cuda.empty_cache() if torch.cuda.is_available() else None
gc.collect()

print(f"Frequent sync: {time_frequent:.2f}s")
print(f"Reduced sync: {time_reduced:.2f}s")
print(f"Speedup: {time_frequent/time_reduced:.2f}x")

---

## ✋ Try It Yourself: Exercise

Profile the following inefficient training loop and identify all bottlenecks:

1. Profile with PyTorch Profiler
2. Identify at least 3 performance issues
3. Fix each issue and measure the improvement

<details>
<summary>💡 Hint</summary>

Look for:
- Data loading issues (num_workers)
- Unnecessary .item() calls
- Missing pin_memory
- Inefficient optimizer.zero_grad()

</details>

In [ ]:
# Inefficient training loop - Find and fix the bottlenecks!
def inefficient_training(model, trainset, epochs=1):
    """This function has multiple performance issues. Can you find them?"""
    
    # Issue 1: ?
    loader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=0)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    
    for epoch in range(epochs):
        for i, (inputs, labels) in enumerate(loader):
            # Issue 2: ?
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Issue 3: ?
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Issue 4: ?
            if i % 1 == 0:
                print(f"Loss: {loss.item():.4f}", end='\r')
            
            if i >= 100:
                break

# YOUR CODE HERE: Profile and fix the issues
# Then create efficient_training() with all fixes applied

---

## Checkpoint

You've learned:
- ✅ Using PyTorch Profiler for detailed analysis
- ✅ Generating and reading Chrome traces
- ✅ Identifying CPU, GPU, and data loading bottlenecks
- ✅ Memory profiling techniques
- ✅ Common optimizations (num_workers, pin_memory, reduced sync)

---

## DGX Spark Tips

With 128GB unified memory:
- Increase `num_workers` (try 4-8)
- Use larger `batch_size` 
- Enable `pin_memory=True`
- Use `persistent_workers=True` for faster epoch starts

---

## Further Reading

- [PyTorch Profiler Tutorial](https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html)
- [CUDA Best Practices](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/)
- [TensorBoard Profiler Plugin](https://pytorch.org/tutorials/intermediate/tensorboard_profiler_tutorial.html)

In [ ]:
# Cleanup
import gc
import shutil

# Set to False if you want to keep trace files for inspection
DELETE_TRACES = True

# Remove trace files (optional - set DELETE_TRACES=False to keep them)
if DELETE_TRACES and trace_dir.exists():
    shutil.rmtree(trace_dir)
    print(f"Removed trace directory: {trace_dir}")
else:
    print(f"Trace files preserved at: {trace_dir}")
    print("To view traces: Open Chrome, go to chrome://tracing, and load the JSON file")

torch.cuda.empty_cache()
gc.collect()

print("Cleanup complete!")